In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import docx
from string import punctuation
import re
import csv 
import os
from simhash import Simhash, SimhashIndex

In [5]:
from src.utils import search_API, fetch_details, create_csv_path, collectFromEndnote, collectFromTables

In [13]:
rba_path = os.path.abspath('data/1. Risicoanalyse kinderformularium gentamicine nieuw.docx')

In [14]:
csv_path = create_csv_path(rba_path)

In [15]:
referenceList = collectFromEndnote(rba_path)

In [78]:
def writeFromEndnoteToCsv(referenceList):
    #Loop over the reference list in the docx file
    for ref in referenceList:
        ref = ref.strip()
        if (ref.lower().startswith('referen')):
            continue
        elif (ref.strip() == ''):
            continue
        else:
            print('')
            search_item = {
                'p_authors': '',
                'p_title': '',
                'p_doi': ''
            }
            x1 = re.findall("(.*et al.?)([^.]+).", ref)
            x2 = re.findall("(.*)\d{4}.?\s?(\".*\")", ref)
            x3 = re.findall(r'doi:?\s?(.+).?', ref)
            
            if x3:
                search_item['p_doi'] = x3[0].strip(punctuation).strip()
                
            if x1:
                search_item['p_authors'] = x1[0][0].strip(punctuation).strip()
                search_item['p_title'] = x1[0][1].strip(punctuation).strip()
                
            elif x2:
                search_item['p_authors'] = x2[0][0].strip(punctuation).strip()
                search_item['p_title'] = x2[0][1].strip(punctuation).strip()

            elif (ref.lower().startswith('who') or ref.lower().startswith('lci') \
                  or ref.lower().startswith('nvn') or ref.lower().startswith('nice')):
                continue

            try:
                search_query = search_item['p_doi'] if search_item['p_doi'] else search_item['p_title'] 
                print('####search_query', search_query)
                papers = searchArticle(search_query)
                referenceFound = False

                for paperIndex, paper in enumerate(papers['PubmedArticle']):
                    paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
                    if (Simhash(paperTitle).distance(Simhash(search_item['p_title'])) <= 5):
                        doi = ''
                        for idn, ids in enumerate(papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList']):
                            if papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn].attributes['IdType'] == 'doi':
                                doi = papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn]

                        abstract = ''
                        if 'Abstract' in paper['MedlineCitation']['Article']:
                            abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])

                        data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                        with open(csv_path, 'a') as csvfile:
                            cw = csv.writer(csvfile, delimiter=',')
                            cw.writerow(data)
                        referenceFound = True 
                        break
            except:
                print('!!!! The reference could not be found automaticaly, please add it to the list manually', ref)

In [80]:
writeFromEndnoteToCsv(referenceList)





####search_query NVK werkboek Infectieziekten bij kinderen
!!!! The reference could not be found automaticaly, please add it to the list manually Van Furth AM et al. NVK werkboek Infectieziekten bij kinderen. 2008

####search_query TDM gentamicine 8-7-2015
!!!! The reference could not be found automaticaly, please add it to the list manually Coenradie S et al. TDM gentamicine 8-7-2015.


####search_query Gentamicin pharmacokinetics and dosing in neonates with hypoxic ischemic  encephalopathy receiving hypothermia
!!!! The reference could not be found automaticaly, please add it to the list manually Frymoyer A et al. Gentamicin pharmacokinetics and dosing in neonates with hypoxic ischemic  encephalopathy receiving hypothermia. Pharmacotherapy. 2013 Jul;33(7):718-26


####search_query Werkboek kindernefrologie 2010
!!!! The reference could not be found automaticaly, please add it to the list manually Kneepkens CMF et al. Werkboek kindernefrologie 2010.

####search_query Drug prescrib

!!!! The reference could not be found automaticaly, please add it to the list manually van Maarseveen, E. M., A. Sprij, and D. J. Touw. 2016. "Extended-Interval Dosing of Gentamicin Aiming for a Drug-Free Period in Neonates: A Prospective Cohort Study."  Ther Drug Monit 38 (3):402-6. doi: 10.1097/ftd.0000000000000283.


In [ ]:
referencesFromTables = collectFromTables()

In [66]:
def writeFromTablesToCsv():
    header = ['record_id', 'title', 'abstract', 'doi', 'final_included']
    with open(csv_path, 'w', newline='') as csvfile:
        cw = csv.writer(csvfile, delimiter=',')
        cw.writerow(header)

    #Loop over the reference list in the docx file
    for ref in referencesFromTables:
        ref = ref['title'].strip()

        if (ref.lower().startswith('reference') or ref.lower().startswith('referen')):
            print('There might be some problem with the reference below. \
                Please check and add the reference if it is not on the list',ref)
            continue
        elif (ref.strip() == ''):
            print('There might be some problem with the reference below. \
                Please check and add the reference if it is not on the list',ref)
            continue
        else:
            try:
                papers = searchArticle(ref)
                referenceFound = False

                for paperIndex, paper in enumerate(papers['PubmedArticle']):
                    paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
                    if (paperTitle == textTitle.strip(punctuation).strip()):
                        doi = ''
                        for idn, ids in enumerate(papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList']):
                            if papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn].attributes['IdType'] == 'doi':
                                doi = papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn]

                        abstract = ''
                        if 'Abstract' in paper['MedlineCitation']['Article']:
                            abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])

                        data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                        with open(csv_path, 'a') as csvfile:
                            cw = csv.writer(csvfile, delimiter=',')
                            cw.writerow(data)
                        referenceFound = True 
                        break

            except:
                print('!!!! The reference could not be found automaticaly, please add it to the list manually')
                print(ref)